In [ ]:
import importlib
import pathlib
import os
import sys
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
ticker="GME"
report_name=f"{ticker.upper()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}_due_diligence"
base_path=os.path.abspath(os.path.join('.'))

In [ ]:
display(Markdown(f"# {datetime.now().strftime('%Y/%m/%d %H:%M:%S')} - {ticker.upper()} - Due Diligence"))

In [ ]:
report_cache_dir = pathlib.Path(base_path, "notebooks", "reports", report_name)
if not os.path.isdir(report_cache_dir):
    print("Reports data directory not found.")
    print(f"Creating {report_cache_dir}")
    os.mkdir(report_cache_dir);
else:
    print(f"Found reports directory {report_cache_dir}")
    print(os.listdir(report_cache_dir));

In [ ]:
from gamestonk_terminal.technical_analysis import trendline_api as trend
from gamestonk_terminal.due_diligence import finviz_view as finviz

In [ ]:
df_stock_cache = pathlib.Path(report_cache_dir, f"{ticker}_stock_data.pkl")
if os.path.isfile(df_stock_cache):
    print(f"Found a cache file. Loading {df_stock_cache}")
    df_stock = pd.read_pickle(df_stock_cache)
else:
    print("Cache file not found. Getting data.")
    df_stock = trend.load_ticker(ticker, (datetime.now() - timedelta(days=180)).strftime("%Y-%m-%d"))
    df_stock = trend.find_trendline(df_stock, "OC_High", "high")
    df_stock = trend.find_trendline(df_stock, "OC_Low", "how")
    print(f"Saving cache file {ticker}_stock_data.pkl")
    df_stock.to_pickle(df_stock_cache)

## Candle View

In [ ]:
import mplfinance as mpf
mc = mpf.make_marketcolors(up='green',down='red',
                           edge='black',
                           wick='black',
                           volume='in',
                           ohlc='i')
s  = mpf.make_mpf_style(marketcolors=mc, gridstyle=":", y_on_right=True)

ap0 = []

if "OC_High_trend" in df_stock.columns:
    ap0.append(
        mpf.make_addplot(df_stock["OC_High_trend"], color="g"),
    )

if "OC_Low_trend" in df_stock.columns:
    ap0.append(
        mpf.make_addplot(df_stock["OC_Low_trend"], color="b"),
    )

In [ ]:
mpf.plot(df_stock,type='candle',mav=(20,50,200),volume=True, addplot=ap0,
         xrotation=0, style=s, figratio=(10,7), figscale=2.00, 
        update_width_config=dict(candle_linewidth=1.0,candle_width=0.8, volume_linewidth=1.0))

## Analyst Targets

In [ ]:
df_fa = finviz.analyst_df(ticker)
print(df_fa.to_string())

In [ ]:
from gamestonk_terminal.fundamental_analysis import av_view as av

## Annual Income Statement

In [ ]:
# Annual Income Statement
av.income_statement([], ticker)

## Quarterly Income Statement

In [ ]:
# Quarterly Income Statement
av.income_statement(["-q"], ticker)

## Annual Balance Sheet

In [ ]:
# Annual Balance Sheet
av.balance_sheet([], ticker)

## Annual Cash Flow

In [ ]:
# Annual cash flow
av.cash_flow([], ticker)

In [ ]:
!jupyter nbconvert {os.path.join("notebooks", "reports", report_name + ".ipynb")} --to html --no-input